In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train_and_test2.csv
/kaggle/input/first-ssh/id_ed25519.pub
/kaggle/input/first-ssh/id_ed25519
/kaggle/input/first-ssh/Bash


In [2]:
!mkdir -p /root/.ssh
!cp /kaggle/input/first-ssh/id_ed25519 /root/.ssh/
!cp /kaggle/input/first-ssh/id_ed25519.pub /root/.ssh/
!chmod 600 /root/.ssh/id_ed25519
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!ssh -T git@github.com


# github.com:22 SSH-2.0-9e814c2a
# github.com:22 SSH-2.0-9e814c2a
# github.com:22 SSH-2.0-9e814c2a
# github.com:22 SSH-2.0-9e814c2a
# github.com:22 SSH-2.0-9e814c2a
Hi TIMAR101/DS_journey! You've successfully authenticated, but GitHub does not provide shell access.


In [3]:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
class DataLoader:
    def __init__(self, path):
        self.path = path
        self.df = None
        
    def load(self):
        try:
            self.df = pd.read_csv(self.path)
            print("✅ Данные загружены")
            print(f"Размер данных: {self.df.shape}")
            return True        

        
        except FileNotFoundError:
            print("❌ Файл не найден")
            return False
    def show_head(self, n = 5):

        if self.df is not None:
            display(self.df.head(n))
        else:
            print("Данные не загруженные! Сначала вызовите метод load!")
    
    def show_info(self):
        if self.df is not None:
            print("\n --- Общие данные об датасете:")
            print(self.df.info())
            print("\n--- Пропущенные значения:")
            print(self.df.isnull())            
            print(self.df.isnull().sum())

        else:
            print("Данные не загруженные! Сначала вызовите метод load!")
    def fix_columns_names(self):

        if self.df is not None:

            corrections = {
                '2urvived': 'Survived',
                'PasengerId': 'PassengerId',  # Пример другой возможной ошибки
                'Fare': 'TicketPrice'         # Пример переименования
            }
            valid_columns = {key:val for key, val in corrections.items() if key in self.df.columns}

            self.df.rename(columns = valid_columns, inplace = True)

            print(f"Переменованы следующие столбцы: {valid_columns}")

        else:
            print("Данные не загруженные! Сначала вызовите метод load!")

    def fill_NA(self):
            self.df['Age'] = self.df['Age'].fillna(self.df['Age'].median())
            self.df['Embarked'] = self.df['Embarked'].fillna(self.df['Embarked'].mode()[0])
            self.df['TicketPrice'] = self.df['TicketPrice'].fillna(self.df['TicketPrice'].median())
            self.df['Sex'] = self.df['Sex'].fillna(self.df['Sex'].mode()[0])

    def fill_cat(self):
            print("Уникальные значения Sex до map:", self.df['Sex'].unique())
            print("Уникальные значения Embarked до map:", self.df['Embarked'].unique())
            self.df['Sex'] = self.df['Sex'].map({'male': 0, 'female': 1})
            self.df['Embarked'] = self.df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
            # Заполняем NaN, которые появились после map
            self.df['Sex'] = self.df['Sex'].fillna(self.df['Sex'].mode()[0])
            self.df['Embarked'] = self.df['Embarked'].fillna(self.df['Embarked'].mode()[0])
            print("NaN после кодирования:", self.df[['Sex', 'Embarked']].isnull().sum())
            print("Уникальные значения Sex после map:", self.df['Sex'].unique())
            print("Уникальные значения Embarked после map:", self.df['Embarked'].unique())


    
       
    def div_tain_test(self):

        print(self.df['Survived'].unique())
        print(self.df['Survived'].isnull().sum())
        features = ['Pclass', 'Sex', 'Age', 'sibsp', 'Parch', 'TicketPrice', 'Embarked']
        X = self.df[features]
        #X = self.df.drop('Survived', axis=1)
        y = self.df['Survived']
        self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    def model_LogReg(self):

        print("Проверка на пропущенные значения:")

        print(self.X_train.isnull().sum())
        print(self.X_valid.isnull().sum())

        #return


        # Обучение модели
        model = LogisticRegression(max_iter=1000)
        model.fit(self.X_train, self.y_train)
        # Предсказания на валидационной выборке
        y_pred = model.predict(self.X_valid)
        # Оценка качества модели
        print("Accuracy:", accuracy_score(self.y_valid, y_pred))
        print("\nClassification report:\n", classification_report(self.y_valid, y_pred))
        print("\nConfusion matrix:\n", confusion_matrix(self.y_valid, y_pred))

    def model_RF(self):

        model = RandomForestClassifier()
        model.fit(self.X_train, self.y_train)
        print(f"Accuracy: {model.score(self.X_valid, self.y_valid):.2f}")

       # self.X_train, self.X_valid, self.y_train, self.y_valid

        




    def plot_survival(self):
        if self.df is not None:

            plt.figure(figsize=(8, 5))
            sns.countplot(x='Survived', data=self.df)  # Обратите внимание на опечатку в названии столбца!
            plt.title('Распределение выживших')
            plt.show()
            sns.countplot(x='Sex', hue='Survived', data=self.df)
            plt.title('Выживаемость по полу')
            plt.show()
            sns.countplot(x='Pclass', hue='Survived', data=self.df)
            plt.title('Выживаемость по классу')
            plt.show()


        else:
            print("Данные не загруженные! Сначала вызовите метод load!")   
            


In [4]:
loader = DataLoader("/kaggle/input/titanic/train_and_test2.csv")
if loader.load():
    print("Truth")

    loader.show_head()
    #loader.show_info()
    loader.fix_columns_names()
    #loader.plot_survival()
    loader.fill_NA()
    #loader.fill_cat()
    loader.div_tain_test()
    loader.model_LogReg()
    loader.model_RF()








✅ Данные загружены
Размер данных: (1309, 28)
Truth


,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0


Переменованы следующие столбцы: {'2urvived': 'Survived', 'Fare': 'TicketPrice'}
[0 1]
0
Проверка на пропущенные значения:
Pclass         0
Sex            0
Age            0
sibsp          0
Parch          0
TicketPrice    0
Embarked       0
dtype: int64
Pclass         0
Sex            0
Age            0
sibsp          0
Parch          0
TicketPrice    0
Embarked       0
dtype: int64
Accuracy: 0.767175572519084

Classification report:
               precision    recall  f1-score   support

           0       0.79      0.92      0.85       189
           1       0.64      0.37      0.47        73

    accuracy                           0.77       262
   macro avg       0.72      0.65      0.66       262
weighted avg       0.75      0.77      0.74       262


Confusion matrix:
 [[174  15]
 [ 46  27]]
Accuracy: 0.71


In [5]:
!mkdir -p /kaggle/working/ssh_keys
!cp /root/.ssh/id_ed25519 /kaggle/working/ssh_keys/
!cp /root/.ssh/id_ed25519.pub /kaggle/working/ssh_keys/
